## Bibliotecas necessárias:
* beautifulsoup4
* lxml   
* pandas
* pdfplumber
* requests
* PyPDF2
* re

## Instalação
```sh 
pip3 install lxml beautifulsoup4 requests pandas pdfplumber
```

In [1]:
from bs4 import BeautifulSoup

import requests
import urllib
import os
import subprocess




Definir constantes

In [15]:
url = 'https://spreadsheets.google.com/feeds/list/1IJBDu8dRGLkBgX72sRWKY6R9GfefsaDCXBd3Dz9PZNs/14/public/values'
nome = 'GRACIANA'

In [16]:
try:
    pasta_de_download
except:
    pasta_de_download = './arquivos_baixados'

Criar lista contendo todos os endereços dos arquivos PDF para download listados no endereço da variável url

In [3]:
def scrape_lista_de_pdf(url):
    pdf_doc = requests.get(url).text
    soup = BeautifulSoup(pdf_doc, 'xml')
    lista_de_pdf = [ item.pdf.text for item in soup.find_all('entry') ]
    return lista_de_pdf

Baixar os arquivos somente se já não existirem localmente

In [4]:
def criar_pasta_de_download(pasta_de_download):
    try:
        if not os.path.isdir(pasta_de_download):
            os.mkdir(pasta_de_download) 
    except OSError:
        print ("Criação do diretório de download falhou %s " % pasta_de_download)

In [5]:
def download_arquivo(url,pasta_de_download='.'):
    nome_do_arquivo = url.rsplit('/', 1)[-1]
    caminho = os.path.join(pasta_de_download, nome_do_arquivo)
    try:
        if not os.path.isfile(caminho):
            wget.download(url, out=caminho)
    except:
        print("Não foi possível salvar o arquivo {}".format(url))

In [6]:
def download_arquivo(url,pasta_de_download='.'):
    nome_do_arquivo = url.rsplit('/', 1)[-1]
    caminho = os.path.join(pasta_de_download, nome_do_arquivo)
    try:
        if not os.path.isfile(caminho):
            arquivo_stream = requests.get(url, stream=True)
            with open(caminho, 'wb') as local_file:
                for data in arquivo_stream:
                    local_file.write(data)
    except:
        print('Não foi possível salvar o arquivo na url {}'.format(url))



In [11]:
def procura_nome_pdfgrep(nome, pasta='.'):
    resultado_da_busca = subprocess.getoutput('pdfgrep -i "{}" {}/*.pdf'.format(nome, pasta))
    return resultado_da_busca 

In [12]:
criar_pasta_de_download(pasta_de_download)

In [13]:
lista_de_links=scrape_lista_de_pdf(url)
for link in lista_de_links:
    download_arquivo(link, pasta_de_download)

Busca usando pdfgrep

In [14]:
print('Procurando agendamento de "{}"'.format(nome))
procura_nome_pdfgrep(nome, pasta_de_download)

Procurando agendamento de "GRACIANA"


'./arquivos_baixados/Drive-Riomar-Kennedy-27.02.2021.pdf:        NELI GRACIANA DE JESUS            DIONISIO TORRES       SHOPPING RIOMAR KENNEDY2021-02-27   13:00:00\npdfgrep: Could not open ./arquivos_baixados/LISTA%20DRIVE%20CASTEL%C3%83O%2008022021.pdf\npdfgrep: Could not open ./arquivos_baixados/LISTA%20DRIVE%20CENTRO%20DE%20EVENTOS%2008022021.pdf'